---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [ ]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [ ]:
def graph_identification():
    #for the submission, purposely comment them out. They are CORRECT CODE
    
    #import matplotlib.pyplot as plt
    
    #for G in P1_Graphs:
        #print("Test_CC,SPL_values:")
        #CC_list = []
        #CC_list.append((nx.average_clustering(G),nx.average_shortest_path_length(G)))
        #print(CC_list)
        #degree = G.degree()
        #degree_values = sorted(set(degree.values()))
        #hist = [list(degree.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        #plt.bar(degree_values,hist)
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.xlabel("Degree")
        #plt.ylabel("Fraction of Nodes")
        #plt.show()
    
    #1,4 are pretty straight forward if the graph is printed (Trend: downward straight line == follow power_law)
    #3 shortest path length is very low compared to other network, which means it is not destroyed by the rewire-process.
    # Maybe due to the parameter k, the number of nearest neighbor, it's hard to judge which one is which; therefore, the
    # best way to distinguish the values is to see the trend of the graph and distribution.
    return ['PA','SW_L','SW_L','PA','SW_H']


---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
def salary_predictions():
    
    #Extract attributes from network : salary and department info
    salary = nx.get_node_attributes(G,'ManagementSalary')
    department = nx.get_node_attributes(G,'Department')
    
    #Build your own dataframe for Machine Learning and add features that you have learned in WEEK3 to it 
    df_salary = pd.DataFrame(list(salary.values()),columns=['mng'])
    dep_df = pd.DataFrame(list(department.values()),columns=['dep'])
    page_department = pd.DataFrame(list(nx.pagerank(G,alpha=0.8).values()),columns=['page'])
    close_cent = pd.DataFrame(list(nx.closeness_centrality(G,normalized=True).values()),columns=['close'])
    hit_hub = pd.DataFrame(list(nx.hits(G)[0].values()),columns=['hub'])
    hit_auth = pd.DataFrame(list(nx.hits(G)[1].values()),columns=['auth'])
    union_df = pd.concat([df_salary,dep_df,page_department,close_cent,hit_hub,hit_auth],axis=1)
    
    
    #import sklearn libraries.
    df = union_df
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split,GridSearchCV
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_auc_score
    
    #split X,y_train and X,y_test and scale them.
    test_index = np.where(np.isnan(df['mng'])==True)
    train_index = np.where(np.isnan(df['mng'])==False)
    scaler = MinMaxScaler()
    
    X_train = df.iloc[train_index].iloc[:,1:]
    X_train_scaled = scaler.fit_transform(X_train)
    y_train = df.iloc[train_index]['mng']

    X_test = df.iloc[test_index].iloc[:,1:]
    X_test_scaled = scaler.fit_transform(X_test)
    y_test = df.iloc[test_index]['mng']
    
    #Find optimal parameters by using GridSearchCV.
    lg = LogisticRegression()
    parameters = {'C':[0.3,0.5,1.0],'class_weight':['balanced',{1:2},{1:10}]}
    clf = GridSearchCV(lg,parameters)
    clf.fit(X_train_scaled,y_train)
    y_predicted = clf.predict_proba(X_test_scaled)
    result = pd.Series(y_predicted[:,1],index=test_index[0])
    return result


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [5]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [16]:
def new_connections_predictions():
    #prediction values of connection nodes that will be used for training data 
    common_neigh = [(e[0],e[1],len(list(nx.common_neighbors(G,e[0],e[1])))) for e in list(future_connections.index)]
    common_neigh_df = pd.DataFrame(list(map(lambda x:x[2],common_neigh)),index=list(map(lambda x:(x[0],x[1]),common_neigh)),columns=['comm'])

    jaccard_coeff = list(nx.jaccard_coefficient(G))
    jaccard_coeff_df = pd.DataFrame(list(map(lambda x:x[2],jaccard_coeff)),index=list(map(lambda x:(x[0],x[1]),jaccard_coeff)),columns=['jac'])
    
    res_index = list(nx.resource_allocation_index(G))
    res_index_df = pd.DataFrame(list(map(lambda x:x[2],res_index)),index=list(map(lambda x:(x[0],x[1]),res_index)),columns=['rai'])
    
    adamic_adar = list(nx.adamic_adar_index(G))
    adamic_adar_df = pd.DataFrame(list(map(lambda x:x[2],adamic_adar)),index=list(map(lambda x:(x[0],x[1]),adamic_adar)),columns=['adar'])
    
    pref_attach = list(nx.preferential_attachment(G))
    pref_attach_df = pd.DataFrame(list(map(lambda x:x[2],pref_attach)),index=list(map(lambda x:(x[0],x[1]),pref_attach)),columns=['pref'])
    
    ra_index_mng = list(nx.ra_index_soundarajan_hopcroft(G,[(e[0],e[1]) for e in list(future_connections.index)],'ManagementSalary'))
    ra_index_mng_df = pd.DataFrame(list(map(lambda x:x[2],ra_index_mng)),index=list(map(lambda x:(x[0],x[1]),ra_index_mng)),columns=['ra_mng'])
    
    ra_index_dep = list(nx.ra_index_soundarajan_hopcroft(G,[(e[0],e[1]) for e in list(future_connections.index)],'Department'))
    ra_index_dep_df = pd.DataFrame(list(map(lambda x:x[2],ra_index_dep)),index=list(map(lambda x:(x[0],x[1]),ra_index_dep)),columns=['ra_dep'])
    
    cn_common_mng = list(nx.cn_soundarajan_hopcroft(G,[(e[0],e[1]) for e in list(future_connections.index)],'ManagementSalary'))
    cn_common_mng_df = pd.DataFrame(list(map(lambda x:x[2],cn_common_mng)),index=list(map(lambda x:(x[0],x[1]),cn_common_mng)),columns=['cn_mng'])
    
    cn_common_dep = list(nx.cn_soundarajan_hopcroft(G,[(e[0],e[1]) for e in list(future_connections.index)],'Department'))
    cn_common_dep_df = pd.DataFrame(list(map(lambda x:x[2],cn_common_dep)),index=list(map(lambda x:(x[0],x[1]),cn_common_dep)),columns=['cn_dep'])
    
    #sort in ascending order.
    for ddf in [common_neigh_df,jaccard_coeff_df,res_index_df,adamic_adar_df,pref_attach_df,ra_index_mng_df,ra_index_dep_df,cn_common_mng_df,cn_common_dep_df]:
        ddf.sort_index(inplace=True)

    #concatenate all of the dataframes above.
    union_df = pd.concat([common_neigh_df,jaccard_coeff_df,res_index_df,adamic_adar_df,pref_attach_df,ra_index_mng_df,ra_index_dep_df,cn_common_mng_df,cn_common_dep_df],axis=1)
    
    #Machine Learning part
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import GridSearchCV
    from sklearn.preprocessing import MinMaxScaler

    #sort the edges info that will be connected (1.0 will be connected and 0.0 will not and NaN are the one needed to be predicted)
    future_df = future_connections.sort_index()
    
    #find out NaN index and use them for splitting train and test data
    nan_index = list(future_connections[np.isnan(future_connections["Future Connection"])==True].index)
    not_nan_index = list(future_connections[np.isnan(future_connections["Future Connection"])==False].index)
    
    #Especially for LogisticRegression and SVM, data need to be scaled properly.
    scaler = MinMaxScaler()

    #the concatenated dataframe above,union_df, need to be splitted in terms of the index of NaN values in future_connections dataframe and index of not NaN values 
    #y_train should be the dataframe splitted in terms of the index of not_nan value in future_connections dataframe.
    #caution 1: due to the scaled values, y_train need to be array type not dataframe. notice "['Future Connection']" followed by future_df.loc[not_nan_index]
    #caution 2: since slicing is done by 'specific index value e.g.(0,2),(0,3)..etc indexing must be done by "loc" not "iloc"
    
    X_train = union_df.loc[not_nan_index]
    X_train_scaled = scaler.fit_transform(X_train)
    y_train = future_df.loc[not_nan_index]['Future Connection']
    X_test = union_df.loc[nan_index]
    X_test_scaled = scaler.fit_transform(X_test)

    #GridSearchCV model optimization (due to huge computational cost, chose LogisticRegression rather than SVC)
    lg = LogisticRegression()
    parameters = {'C':[0.3,0.5,1.0],'class_weight':['balanced',{1:2},{1:10}]}
    clf = GridSearchCV(lg,parameters)
    clf.fit(X_train_scaled,y_train)
    y_prob = clf.predict_proba(X_test_scaled)[:,1]
    result = pd.Series(y_prob,index = nan_index)

    return result

new_connections_predictions()

(107, 348)    0.069917
(542, 751)    0.022130
(20, 426)     0.913541
(50, 989)     0.022171
(942, 986)    0.022215
(324, 857)    0.022165
(13, 710)     0.484497
(19, 271)     0.439879
(319, 878)    0.022185
(659, 707)    0.022134
(49, 843)     0.022198
(208, 893)    0.022144
(377, 469)    0.020417
(405, 999)    0.041495
(129, 740)    0.038455
(292, 618)    0.051021
(239, 689)    0.022188
(359, 373)    0.021134
(53, 523)     0.266824
(276, 984)    0.022196
(202, 997)    0.022204
(604, 619)    0.192206
(270, 911)    0.022187
(261, 481)    0.214949
(200, 450)    0.999993
(213, 634)    0.022117
(644, 735)    0.151339
(346, 553)    0.022014
(521, 738)    0.021813
(422, 953)    0.043365
                ...   
(672, 848)    0.022187
(28, 127)     0.992709
(202, 661)    0.021996
(54, 195)     0.999997
(295, 864)    0.022159
(814, 936)    0.022123
(839, 874)    0.022215
(139, 843)    0.022162
(461, 544)    0.021671
(68, 487)     0.021646
(622, 932)    0.022149
(504, 936)    0.040654
(479, 528) 